# Title

## read data

In [234]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


target_col = 'SalePrice'
data_folder = "./data/"
# dropping exterior cols bc there is unexpexted value in them, maybe later 
# will do something with them
# electrical have nan that i cant replace or delete for some reason wtf
drop_cols = ['Exterior1st', 'Exterior2nd', 'Id', 'Electrical']
df_test = pd.read_csv(data_folder + 'test.csv').drop(columns=drop_cols, axis =1)
df = pd.read_csv(data_folder + 'train.csv').drop(columns=drop_cols, axis =1)


# replacing null values

In [235]:
import numpy as np
# df['Electrical'] = df['Electrical'].replace(np.nan, pd.NA).dropna()
not_numeric_cols =df.select_dtypes(exclude = ['number']).columns
na_value_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 
                 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
df[na_value_cols] = df[na_value_cols].fillna('NA')
df[not_numeric_cols].isnull().any()
df['MasVnrType'] = df['MasVnrType'].fillna('NA')
numeric_cols = df.select_dtypes(include = ['number']).columns
# for col in df[numeric_cols].columns:
# 	print("{0} var: {1}".format(col, df[col].var(ddof=1)))
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].mean(skipna=True))
print(numeric_cols)

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


In [236]:
print(df.loc[:, df.isna().any()])

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[1460 rows x 0 columns]


## encode_data

## analise features

In [71]:
# check if dataset have unclear categorial types
categorial_cols = []
for col in df.columns:
    if df[col].unique().__len__() < 30:
        categorial_cols.append(col)
        
# print(categorial_cols)
not_numeric_types = df.select_dtypes(exclude = ['float64', 'int64']).columns
for elem in categorial_cols:
    if elem not in not_numeric_types:
        print(elem)
print(df['OverallQual'].unique())


MSSubClass
OverallQual
OverallCond
LowQualFinSF
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageCars
3SsnPorch
PoolArea
MiscVal
MoSold
YrSold
[ 7  6  8  5  9  4 10  3  1  2]


In [ ]:
from matplotlib import pyplot as plt

plt.hist(df['MSSubClass'])

In [45]:
# check if there categorical features listed as numeric 

categorial_cols = []
for col in df.columns:
    if df[col].unique().__len__() < 20:
        categorial_cols.append(col)
not_numeric_types = df.select_dtypes(exclude = ['float64', 'int64']).columns
arr = [elem for elem in categorial_cols if elem not in not_numeric_types]
print(arr)

['MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'PoolArea', 'MoSold', 'YrSold']


## actual encoding


In [237]:
import json
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

json_place = './data/data_description.json'
with open(json_place) as f:
    descr = json.load(f)

not_numeric_types = df.select_dtypes(exclude = ['float64', 'int64']).columns
# not_numeric_types.append('MSSubClass')
encoded = df
ohe_cols = []
ohe_cg = []
for col in not_numeric_types:
    categories = list(descr[col].keys())
    if col == 'MSZoning':
        target_index = categories.index("C")
        categories[target_index] = "C "
    

    # print(col)
    # print(categories)
    
    if categories.__len__() > 7:
        le = LabelEncoder()
        le.fit(categories)
        encoded[col] = le.transform(encoded[col])
    else:
        ohe_cols.append(col)
        ohe_cg.append(categories)

encoder = OneHotEncoder(categories = ohe_cg, handle_unknown='ignore',sparse_output=False)
ohe_encoded = encoder.fit_transform(df[ohe_cols])
ohe_encoded_df = pd.DataFrame(ohe_encoded, columns=encoder.get_feature_names_out())
encoded_df = pd.concat([encoded.drop(columns=ohe_cols, axis=1), ohe_encoded_df], axis=1)



# analise data

## удалить выбросы


In [242]:
from scipy import stats
import numpy as np

# numeric_cols.drop(labels='SalePrice')
df_zscore = stats.zscore(encoded_df[numeric_cols])
encoded_df['z_score'] = df_zscore.max(axis=1)
# print(encoded_df['z_score'])
# print(df_zscore)
encoded_df[encoded_df['z_score'] < 7].shape
df_clear_z = encoded_df[encoded_df['z_score'] < 5].drop(columns=['z_score'], axis=1)


## Графики feature - target

In [ ]:
from matplotlib import pyplot as plt
print(numeric_cols)
feature = 'LotArea'
# df_clear_z = encoded_df[encoded_df['z_score'] < 5]
Y = df_clear_z[target_col]
X = df_clear_z[feature]

plt.scatter(X, Y)
plt.xlabel(feature)
plt.ylabel(target_col)

In [ ]:
# Определяем количество признаков
features = numeric_cols  # Все колонки, кроме последней (target)
num_features = len(numeric_cols)

# Создаем подграфики
fig, axes = plt.subplots(nrows=num_features, ncols=1, figsize=(8, 4 * num_features))

# Строим графики
for i, feature in enumerate(features):
    axes[i].scatter(df[feature], df[target_col], color='blue')
    axes[i].set_title(f'{feature} vs Target')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Target')

# Автоматическая настройка макета
plt.tight_layout()
plt.show()

## train test model

In [37]:
from sklearn import metrics
model = RandomForestRegressor(criterion='friedman_mse', random_state=42, n_estimators=100)
model.fit(X_train_clf, Y_train_clf)

y_pred = model.predict(X_test_clf)
mse = metrics.mean_absolute_error(y_true=Y_test_clf, y_pred=y_pred)
print("mse = {0}".format(mse))
print(df[target_col])
print(mse / df[target_col].mean() )

mse = 17759.88623287671
0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64
0.09816365708545488
